# Valuing Caps and Floors

In [32]:
# This program is distributed in the hope that it will be useful, but WITHOUT
# ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
# FOR A PARTICULAR PURPOSE.  See the FinancePy license for more details.

We value caps and floors using Black's model

In [33]:
import numpy as np
import matplotlib.pyplot as plt

In [34]:
from financepy.finutils import *
from financepy.products.libor import *
from financepy.market.curves import *

## Building a Libor Curve

In [35]:
valuationDate = FinDate(6, 6, 2018)

In [36]:
spotDays = 0
settlementDate = valuationDate.addWorkDays(spotDays)
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA

depos = []
depo = FinLiborDeposit(settlementDate, "1M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "2M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0230, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "9M", 0.0230, depoDCCType); depos.append(depo)

In [37]:
accrual = FinDayCountTypes.THIRTY_360
freq = FinFrequencyTypes.SEMI_ANNUAL
longEnd = FinDateGenRuleTypes.BACKWARD

spotDays = 2
settlementDate = valuationDate.addWorkDays(spotDays)

swaps = []
swap = FinLiborSwap(settlementDate, "1Y", 0.0250, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "2Y", 0.0255, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "3Y", 0.0260, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", 0.0265, freq, accrual); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", 0.0270, freq, accrual); swaps.append(swap)

In [10]:
liborCurve = FinLiborCurve("USD_LIBOR", settlementDate, depos, [], swaps)

## Creating a Cap and Floor

In [11]:
capType = FinLiborCapFloorType.CAP
floorType = FinLiborCapFloorType.FLOOR

In [12]:
strikeRate = 0.02

In [13]:
cap = FinLiborCapFloor(settlementDate, "2Y", capType, strikeRate)
flr = FinLiborCapFloor(settlementDate, "2Y", floorType, strikeRate)

In [14]:
print(cap)

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorType.CAP
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


In [15]:
print(flr)

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorType.FLOOR
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


## Valuation

### Black's Model

We start with Black's model with 25% volatility

In [16]:
model = FinLiborModelBlack(0.25)

In [17]:
cap.value(valuationDate, liborCurve, model)

11374.986044088717

In [18]:
flr.value(valuationDate, liborCurve, model)

839.149738252386

In [19]:
cap.print()

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
STRIKE COUPON: 2.0
OPTION TYPE: FinLiborCapFloorType.CAP
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA


In [20]:
cap.printLeg()

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
OPTION TYPE FinLiborCapFloorType.CAP
STRIKE (%): 2.0
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE WED 6 JUN 2018
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    CAPLET_PV       CUM_PV
 FRI 8 JUN 2018  0.0000000    0.00000         0.00     1.000000         0.00         0.00
MON 10 SEP 2018  0.2555556    2.34863       885.63     0.994034       885.63       885.63
MON 10 DEC 2018  0.2500000    2.28739       710.13     0.988382       761.57      1647.21
 FRI 8 MAR 2019  0.2444444    2.28951       695.58     0.982881       814.74      2461.94
MON 10 JUN 2019  0.2555556    3.03491      2579.49     0.975316      2593.31      5055.26
 MON 9 SEP 2019  0.2472222    2.60810      1456.85     0.969068      1559.08      6614.34
 MON 9 DEC 2019  0.2500000    2.57912      1394.03     0.962860      1545.36      8159.70
 MON 9 MAR 2020  0.2500000    2.57912      1385.10     

## SABR Model

In [24]:
alpha = 0.037; beta = 0.5; rho  = 0.1; nu = 0.573

In [25]:
model = FinLiborModelSABR(alpha, beta, rho, nu)

In [26]:
cap.value(valuationDate, liborCurve, model)

11477.597613794864

In [29]:
flr.value(valuationDate, liborCurve, model)

941.7613079585357

In [30]:
flr.printLeg()

START DATE: FRI 8 JUN 2018
MATURITY DATE: MON 8 JUN 2020
OPTION TYPE FinLiborCapFloorType.FLOOR
STRIKE (%): 2.0
FREQUENCY: FinFrequencyTypes.QUARTERLY
DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE WED 6 JUN 2018
PAYMENT_DATE     YEAR_FRAC   FWD_RATE    INTRINSIC           DF    FLRLET_PV       CUM_PV
 FRI 8 JUN 2018  0.0000000    0.00000         0.00     1.000000         0.00         0.00
MON 10 SEP 2018  0.2555556    2.34863         0.00     0.994034         0.00         0.00
MON 10 DEC 2018  0.2500000    2.28739         0.00     0.988382        54.55        54.55
 FRI 8 MAR 2019  0.2444444    2.28951         0.00     0.982881       126.65       181.20
MON 10 JUN 2019  0.2555556    3.03491         0.00     0.975316        19.05       200.25
 MON 9 SEP 2019  0.2472222    2.60810         0.00     0.969068       114.45       314.70
 MON 9 DEC 2019  0.2500000    2.57912         0.00     0.962860       169.74       484.44
 MON 9 MAR 2020  0.2500000    2.57912         0.00   

Copyright (c) 2020 Dominic O'Kane